### 开发者笔记
作者：***HakureiPOI***

最后更新：2024/8/19

功能：根据关键词对ESJ轻小说进行下载

待实现功能：一大堆

使用手册：见实例化部分

---

### 准备工作

In [ ]:
!pip install opencc

In [ ]:
import pandas as pd
import requests
import logging
import json
import time
import os
import sys
import re
import bs4
import pickle
from opencc import OpenCC
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [ ]:
os.makedirs('logs', exist_ok = True)

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - [%(levelname)s] - %(message)s')

streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
logger.addHandler(streamHandler)

fileHandler = logging.FileHandler('logs/log.txt')
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)

---
### 轻小说爬虫类

以后应该都会按照这个模板来实现

In [ ]:
class AbstractScraper():
    def __init__(self, usr, pwd, domain_name, path):
        self.usr = usr
        self.pwd = pwd
        self.domain_name = domain_name
        self.path = path
        self.api = Interface(usr, pwd, domain_name)
        self.keyword = ''
        self.novels = []
        self.simplify = False
        self.login()
    def login(self):
        pass
    def search(self):
        pass
    def print_novels(self):
        pass
    def set_keyword(self, keyword):
        pass
    def set_simplify(self):
        pass
    def get_web_novels(self):
        pass
    def get_local_novels(self):
        pass
    def update_novels(self):
        pass
    def download_novels(self):
        pass
    def save_novels(self):
        pass

In [ ]:
class Scraper(AbstractScraper):
    def __init__(self, usr, pwd, domain_name, path):
        super().__init__(usr, pwd, domain_name, path)

    def login(self):
        try:
            response = self.api.login()
            assert response.status_code == 200
            logger.info(f'login success, {self.usr}')
        except Exception as e:
            logger.warning(f'login error, {type(e).__name__}-{e}')

    def set_keyword(self, keyword):
        try:
            self.keyword = str(keyword)
            assert isinstance(self.keyword, str)
            logger.info(f'set keyword to {self.keyword}')

        except Exception as e:
            logger.warning(f'set keyword error, {type(e).__name__}-{e}')

    def set_simplify(self, simplify):
        try:
            self.simplify = bool(simplify)
            logger.info(f'set simplify to {self.simplify}')

        except Exception as e:
            logger.warning(f'set simplify error, {type(e).__name__}-{e}')

    def search(self, novel_type = 0, order_method = 1):
        try:
            response = self.api.search(self.keyword, novel_type, order_method)
            assert response.status_code == 200
            logger.info(f'search success, {self.keyword}')
            return response
        except Exception as e:
            logger.warning(f'search error, {type(e).__name__}-{e}')

    def print_novels(self):
        try:
            print('novels are as follows:')
            for index, novel in enumerate(self.novels):
                print(f'{index + 1 :<4} : {novel}')
            print(f'altogether, there are {len(self.novels)} novels.')

        except Exception as e:
            logger.warning(f'print novels error, {type(e).__name__}-{e}')

    def get_web_novels(self):
        try:
            self.novels.clear()
            novel_type, order_method = 0, 1
            response = self.search(novel_type, order_method)
            assert response.status_code == 200

            current_page = 1
            soup = BeautifulSoup(response.text, 'html.parser')
            script = soup.find_all('script')[-2].text
            match = re.search(r"total:\s(\d+),", script)
            if match:
                total_pages = int(match.group(1))

            while True:
                novel_list = soup.find_all('div', class_ = 'card-body')
                for novel in novel_list:
                    url = novel.find('a')['href']
                    bookname = novel.find('h5', class_ = 'card-title').text.strip()
                    author = novel.find('div', class_ = 'card-author').text.strip()
                    self.novels.append(Novel(url, bookname, author))

                current_page += 1
                if current_page > total_pages:
                    break

                next_page_url = f'/tags-{novel_type}{order_method}/{self.keyword}/{current_page}.html'
                response = self.api.open(next_page_url)
                assert response.status_code == 200
                soup = BeautifulSoup(response.text, 'html.parser')

                logger.info(f'get page success, next page url : {next_page_url}')

            logger.info(f'get novels success, {len(self.novels)} novels')

        except Exception as e:
            logger.warning(f'get novels error, {type(e).__name__}-{e}')

    def get_local_novels(self, filter = False):
        try:
            self.novels.clear()
            for bookname in os.listdir(self.path):
                novel = Novel('', '', '')
                novel.load(f'{self.path}/{bookname}/{bookname}.pkl')
                if filter:
                    detail = novel.bookname + novel.author + novel.introduction + ''.join(novel.tags)
                    match = re.search(self.keyword, detail)
                    if match:
                        self.novels.append(novel)
                else:
                    self.novels.append(novel)

            logger.info(f'get local novels success, {len(self.novels)} novels')

        except Exception as e:
            logger.warning(f'get local novels error, {type(e).__name__}-{e}')

    def update_novels(self):
        try:
            if self.novels == []:
                logger.warning('novels is empty')
                return

            with ThreadPoolExecutor(max_workers = 50) as executor:
                executor.map(self.__update_helper, self.novels)

            logger.info(f'update novels success, {len(self.novels)} novels')

        except Exception as e:
            logger.warning(f'update novels error, {type(e).__name__}-{e}')

    def update_1_novel(self):
        try:
            if self.novels == []:
                logger.warning('novels is empty')
                return

            self.print_novels()

            seleted = int(input('select novel to update : '))
            while seleted < 1 or seleted > len(self.novels):
                print('invalid input')
                seleted = int(input('select novel to update : '))

            novel = self.novels[seleted - 1]
            self.__update_helper(novel)

        except Exception as e:
            logger.warning(f'update 1 novel error, {type(e).__name__}-{e}')

    def __update_helper(self, novel):
        try:
            response = self.api.open(novel.url)
            assert response.status_code == 200
            soup = BeautifulSoup(response.text, 'html.parser')

            last_update = soup.find('strong', string = '更新日期:').parent.text
            match = re.search(r'(\d{4}-\d{2}-\d{2})', last_update)
            if match:
                last_update = match.group(1)

            if last_update != novel.last_update:
                novel.last_update = last_update
                novel.introduction = soup.find('div', class_ = 'description').text.strip()
                novel.tags = [tag.text for tag in soup.find_all('a', class_ = 'tag')]
                chapterList = soup.find_all('a', {'data-title' : re.compile(r'.+')})

                for index, chapter in enumerate(chapterList):
                    title = chapter['data-title']
                    url = chapter['href']
                    novel.chapters.loc[index] = [index, title, url]

                self.download_novels(novel)

                logger.info(f'update novel success, {novel}')

            else:
                logger.info(f'novel not updated, {novel}')

        except Exception as e:
            logger.warning(f'update helper error, {novel} : {type(e).__name__}-{e}')

    def download_novels(self, novel):
        try:
            for index in range(len(novel.chapters)):
                title = f'第 {index + 1} 章 - {novel.chapters.loc[index, "title"]}\n'
                novel.text.append(title)

                response = self.api.direct_open(novel.chapters.loc[index, 'url'])
                assert response.status_code == 200
                soup = BeautifulSoup(response.text, 'html.parser')
                passages = soup.find('div', class_ = re.compile(r'forum'))

                if passages is None:
                    continue
                else:
                    passages = passages.find_all('p', string = re.compile(r'.+'))

                for p in passages:
                    novel.text.append(p.text)

                logger.info(f'download chapter success, {novel.bookname} : {title[:-1]}')

            novel.save(self.path, self.simplify)

        except Exception as e:
            logger.warning(f'download novels error, {type(e).__name__}-{e}')

    def save_novels(self):
        try:
            for novel in self.novels:
                novel.save(self.path)
                logger.info(f'save novel success, {novel}')

        except Exception as e:
            logger.warning(f'save novels error, {type(e).__name__}-{e}')

---
### 小说类


In [ ]:
class Novel():
    def __init__(self, url, bookname, author):
        self.url = url
        self.bookname = bookname
        self.author = author
        self.last_update = ''
        self.introduction = ''
        self.tags = []
        self.chapters = pd.DataFrame(columns = ['index', 'title', 'url'])
        self.text = []

    def __str__(self):
        return f'{self.bookname} - {self.author}'

    def load(self, path):
        try:
            with open(path, 'rb') as f:
                novel = pickle.load(f)
                self.__dict__.update(novel.__dict__)

        except Exception as e:
            logger.warning(f'load novel error, {type(e).__name__}-{e}')

    def save(self, path, simplify = False):
        try:
            os.makedirs(f'{path}/{self.bookname}', exist_ok = True)

            with open(f'{path}/{self.bookname}/{self.bookname}.pkl', 'wb') as f:
                pickle.dump(self, f)

            with open(f'{path}/{self.bookname}/{self.bookname}.txt', 'w', encoding = 'utf-8') as f:
                f.write(self.bookname + '\n')
                f.write(self.author + '\n')
                f.write(self.introduction + '\n')
                f.write('\n'.join(self.text))

            if simplify:
                with open(f'{path}/{self.bookname}/{self.bookname}.txt', 'r', encoding = 'utf-8') as f:
                    text = f.read()
                    cc = OpenCC('t2s')
                    simplified_text = cc.convert(text)
                    with open(f'{path}/{self.bookname}/{self.bookname}（简中）.txt', 'w', encoding = 'utf-8') as f:
                        f.write(simplified_text)

            logger.info(f'save novel success, {self}')

        except Exception as e:
            logger.warning(f'save novel error, {type(e).__name__}-{e}')

---
### 接口类

主要完成获取 response 的功能

In [ ]:
class Interface():
    def __init__(self, usr, pwd, domain_name):
        self.usr = usr
        self.pwd = pwd
        self.domain_name = domain_name
        self.session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(pool_connections = 100, pool_maxsize = 100)
        self.session.mount('http://', adapter)
        self.session.mount('https://', adapter)
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
        }

    def __post(self, url, data, retries = 3):
        for i in range(retries):
            try:
                response = self.session.post(url, data = data, headers = self.headers)
                response.encoding = response.apparent_encoding
                response.raise_for_status()
                return response
            except Exception as e:
                logger.warning(f'requests post error on attempt {i + 1}, {type(e).__name__}-{e}')
                time.sleep(1)

    def __get(self, url, retries = 3):
        for i in range(retries):
            try:
                response = self.session.get(url, headers = self.headers)
                response.encoding = response.apparent_encoding
                response.raise_for_status()
                return response
            except Exception as e:
                logger.warning(f'requests get error on attempt {i + 1}, {type(e).__name__}-{e}')
                time.sleep(1)

    def login(self):
        url = f'{self.domain_name}inc/mem_login.php'
        data = {
            'email': self.usr,
            'pwd': self.pwd,
            'remember_me' : 'on'
        }
        return self.__post(url, data)

    def search(self, keyword, novel_type, order_method):
        url = f'{self.domain_name}tags-{novel_type}{order_method}/{keyword}/'
        return self.__get(url)

    def open(self, url):
        url = f'{self.domain_name}{url}'
        return self.__get(url)

    def direct_open(self, url):
        return self.__get(url)

---
### 实例化

***使用方法：***
* Step1: 使用 **usr, pwd, domain_name, path** 实例化一个 Scraper 对象

|<font size='3'>变量名</font>|<font size='4'>含义</font>|
|---|---|
|<font size='3'>usr</font>|<font size='3'>你的用户名（ESJ是邮箱登录）</font>|
|<font size='3'>pwd</font>|<font size='3'>你的密码</font>|
|<font size='3'>domain_name</font>|<font size='3'>ESJ网站域名（https://www.esjzone.cc/）</font>|
|<font size='3'>path</font>|<font size='3'>爬取的数据保存地址</font>|

* Step2: 调用 ***scraper.set_keyword()*** 方法指定搜索关键词（不指定的情况下默认全部）

* Step3: *（可选）*调用 ***scraper.set_simplify()*** 方法来指定是否自动下载简中副本

* Step4: 调用 ***scraper.get_web_novels()*** 或者 ***scraper.get_local_novels()*** 获得小说列表

* Step5: *（可选）*调用 ***print_novels*** 查看现在爬虫获取的轻小说列表

(***scraper.get_local_novels()*** 可以指定参数 **filter** 来决定是否使用关键词对本地小说进行筛选)

* Step6: 调用 ***scraper.update_novels()*** 或 ***scraper.update_1_novel()*** 来对列表中全部小说或者指定小说进行更新

* Step7: 如果使用 **Colab** 运行此程序，可以从左侧的文件中自行下载需要的文件或者自行搭载 **Google Drive** 保存文件  

以下是一个简单的实例:

In [ ]:
usr = 'your_username'
pwd = 'your_password'
domain_name = 'https://www.esjzone.cc/'
path = 'novels'

In [ ]:
os.makedirs(path, exist_ok = True)

In [ ]:
scraper = Scraper(usr, pwd, domain_name, path)

In [ ]:
scraper.set_keyword('keyword')

In [ ]:
scraper.set_simplify(True)

In [ ]:
scraper.get_web_novels()

In [ ]:
scraper.print_novels()

In [ ]:
scraper.update_1_novel()